In [ ]:
import pymysql
import numpy as np
import skfuzzy as fuzz

# Database connection details
DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASS = ''
DB_NAME = 'StudentEvaluation'

def calculate_individual_score(count):
    """
    Calculate individual score and return as a label with a percentage.
    """
    x_counts = np.arange(0, 11, 1)
    # Membership functions
    poor = fuzz.trimf(x_counts, [0, 0, 2])
    moderate = fuzz.trimf(x_counts, [1, 3, 5])
    good = fuzz.trimf(x_counts, [4, 6, 8])
    excellent = fuzz.trimf(x_counts, [7, 10, 10])
    
    # Fuzzy membership values
    poor_score = fuzz.interp_membership(x_counts, poor, count)
    moderate_score = fuzz.interp_membership(x_counts, moderate, count)
    good_score = fuzz.interp_membership(x_counts, good, count)
    excellent_score = fuzz.interp_membership(x_counts, excellent, count)
    
    max_score = max(poor_score, moderate_score, good_score, excellent_score)
    percentage = int((count / 10) * 100)
    
    if max_score == excellent_score:
        return f"Excellent ({percentage}%)", percentage
    elif max_score == good_score:
        return f"Good ({percentage}%)", percentage
    elif max_score == moderate_score:
        return f"Moderate ({percentage}%)", percentage
    elif max_score == poor_score:
        return f"Poor ({percentage}%)", percentage
    else:
        return f"Unacceptable ({percentage}%)", percentage

def calculate_overall_fuzzy_score(category_scores):
    """
    Apply fuzzy logic to calculate an overall score based on individual category scores.
    """
    x_scores = np.arange(0, 101, 1)
    # Define membership functions for overall score
    poor = fuzz.trimf(x_scores, [0, 0, 30])
    moderate = fuzz.trimf(x_scores, [20, 50, 70])
    good = fuzz.trimf(x_scores, [60, 80, 90])
    excellent = fuzz.trimf(x_scores, [80, 100, 100])
    
    # Map categorical scores to numerical values for fuzzy processing
    score_mapping = {
        "Excellent": 90,
        "Good": 70,
        "Moderate": 50,
        "Poor": 30,
        "Unacceptable": 10
    }
    
    # Convert individual category scores to corresponding numeric values
    scores_numeric = [score_mapping.get(score.split()[0], 10) for score in category_scores]
    average_score = np.mean(scores_numeric)
    
    # Calculate fuzzy membership for the average score
    poor_level = fuzz.interp_membership(x_scores, poor, average_score)
    moderate_level = fuzz.interp_membership(x_scores, moderate, average_score)
    good_level = fuzz.interp_membership(x_scores, good, average_score)
    excellent_level = fuzz.interp_membership(x_scores, excellent, average_score)
    
    # Determine the highest membership and return the overall classification
    max_level = max(poor_level, moderate_level, good_level, excellent_level)
    
    if max_level == excellent_level:
        return f"Excellent ({int(average_score)}%)", int(average_score)
    elif max_level == good_level:
        return f"Good ({int(average_score)}%)", int(average_score)
    elif max_level == moderate_level:
        return f"Moderate ({int(average_score)}%)", int(average_score)
    elif max_level == poor_level:
        return f"Poor ({int(average_score)}%)", int(average_score)
    else:
        return "Unacceptable (0%)", 0

def process_data_fuzzy():
    # Connect to the database
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME)
    cursor = conn.cursor(pymysql.cursors.DictCursor)

    # Fetch data from student_data table
    cursor.execute("SELECT * FROM student_data")
    rows = cursor.fetchall()

    categories = [
        "Soft Skills - Communication", "Soft Skills - Leadership", 
        "Project", "Sports", "Volunteer Work", 
        "Organization/Clubs and positions", "Events/Programs/Workshops attended", 
        "Leadership Roles"
    ]

    for row in rows:
        name = row['Name']
        email = row['Email']
        telephone = row['Telephone']

        category_scores = []
        numeric_scores = []

        for category in categories:
            # Process each category
            if row.get(category):
                # Extract items and calculate score
                items = [item.strip() for item in str(row[category]).split(",") if item.strip()]
                count = len(items)
                score, percentage = calculate_individual_score(count)
            else:
                items = []
                count = 0
                score, percentage = "Poor (0%)", 0

            category_scores.extend([", ".join(items), count, score])
            numeric_scores.append(percentage)

        # Calculate the overall score using fuzzy logic
        overall_score, overall_percentage = calculate_overall_fuzzy_score(
            [score for i, score in enumerate(category_scores) if i % 3 == 2]
        )

        # Insert or update processed data into the database
        insert_values = [name, email, telephone] + category_scores + [overall_score]
        cursor.execute(
            f"""
            INSERT INTO processed_scores (
                Name, Email, Telephone, 
                `Soft Skills - Communication`, `Soft Skills - Communication Count`, `Soft Skills - Communication Score`,
                `Soft Skills - Leadership`, `Soft Skills - Leadership Count`, `Soft Skills - Leadership Score`,
                `Project`, `Project Count`, `Project Score`,
                `Sports`, `Sports Count`, `Sports Score`,
                `Volunteer Work`, `Volunteer Work Count`, `Volunteer Work Score`,
                `Organization/Clubs and positions`, `Organization/Clubs and positions Count`, `Organization/Clubs and positions Score`,
                `Events/Programs/Workshops attended`, `Events/Programs/Workshops attended Count`, `Events/Programs/Workshops attended Score`,
                `Leadership Roles`, `Leadership Roles Count`, `Leadership Roles Score`,
                `Overall Score`
            ) VALUES ({", ".join(["%s"] * len(insert_values))})
            ON DUPLICATE KEY UPDATE 
                `Overall Score` = VALUES(`Overall Score`)
            """,
            insert_values
        )
        conn.commit()

    print("Data processed successfully and Overall Scores stored!")
    cursor.close()
    conn.close()

# Run the process
process_data_fuzzy()
